# Single Heart Rate Monitor Example

This is a minimal demonstration of how to set up and use BleakFSM.

This examples uses Bluetooth Low Energy (BLE) Heart Rate service.
+ It must support Bluetooth, not just ANT+
+ The GATT Characteristic required is: `00002a37-0000-1000-8000-00805f9b34fb`
+ You may use the [`bluetooth_dissect.py` tool](https://github.com/tensorturtle/bluetooth_dissect) to check if your device supports that GATT Characteristic.

Example compatible devices:
+ Magene H603
+ Wahoo TICKR & TICKR Fit
+ WHOOP 4.0 (Broadcast Heart Rate ON)
+ and many more advertised as "supports Bluetooth Heart Rate"

If you don't have access to a BLE Heart Rate device or want to a different BLE GATT characteristic, see the [Migration Guide](migration_guide.ipynb)


## How to run this Notebook

+ Make sure your computer's bluetooth adapter is powered on.
+ At first, I recommend stepping through each cell, reading the code and explanations.
+ You will need to set your Heart Rate monitor's address the first time.
+ At any time, `print(model.state)` to confirm which state you're in. This will help you figure out what you can do from there.

In [4]:
!pip3 install -r -q requirements.txt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '-q'
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# Setup

In [5]:
import asyncio

In [6]:
from bleak_fsm import machine, BleakModel

In [7]:
model = BleakModel(
    # the following kwargs are optional. Their defaults are:
    scan_stale_time=120, # seconds
    auto_rescan=True,
    auto_rescan_timeout=3 # seconds
)
machine.add_model(model)

Toggle this `USE_PYCYCLING` flag to see how this library can accomodate using standard, raw BLeakClient or Pycycling-wrapped BleakClient.

In [8]:
USE_PYCYCLING = False

In [9]:
# Define callbacks

if USE_PYCYCLING:
    from pycycling.heart_rate_service import HeartRateService
    model.wrap = lambda client: HeartRateService(client)
    model.enable_notifications = lambda client: client.enable_hr_measurement_notifications()
    model.disable_notifications = lambda client: client.disable_hr_measurement_notifications()
    def handle_hr_measurement(value):
        print("Using Pycycling wrapper around BleakClient")
        print(f"Heart Rate: {value}")
    model.set_measurement_handler = lambda client: client.set_hr_measurement_handler(handle_hr_measurement)
else:
    HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID = "00002a37-0000-1000-8000-00805f9b34fb"
    model.enable_notifications = lambda client: client.start_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID, handle_hr_measurement)
    model.disable_notifications = lambda client: client.stop_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID)
    def handle_hr_measurement(sender, data):
        print("Using raw BleakClient")
        print(f"Data received from {sender}: {data}")
        flag = data[0]
        heart_rate = data[1]
        print(f"Heart Rate: {heart_rate} beats per minute")
    model.set_measurement_handler = lambda client: handle_hr_measurement


# Usage

Since we're in a Jupyter notebook, we need to use `create_task()` instead of `get_event_loop().run_until_complete()` to use the existing event loop which Jupyter runs on.

In [10]:
asyncio.create_task(model.start_scan())

<Task pending name='Task-5' coro=<AsyncEvent.trigger() running at /Users/tensorturtle/Library/Python/3.9/lib/python/site-packages/transitions/extensions/asyncio.py:166>>

We could have used

```
await model.start_scan()
```

but it would block until the scan is finished.
For this demonstratoin, we don't want that because we want to demonstrate how `BleakModel.bt_devices` gets populated as the scanning progresses.

In [11]:
model.state

'Scanning'

In [12]:
# Stop scan after delay
async def stop_after_delay(delay, model):
    print("Starting sleep for 2 seconds (giving the scanner time to scan)")
    await asyncio.sleep(delay)
    print("Slept for 2 seconds (sending end signal to scanner)")
    await model.stop_scan()
    print("Stopped scanning")
    

asyncio.create_task(stop_after_delay(2, model))

<Task pending name='Task-7' coro=<stop_after_delay() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_34388/1686944154.py:2>>

In [13]:
async def print_as_we_go():
    for i in range(20):
        print(f"{len(BleakModel.bt_devices)} devices found in {round(i*0.05,2)} seconds.")
        await asyncio.sleep(0.05)

await print_as_we_go()

49 devices found in 0.0 seconds.
49 devices found in 0.05 seconds.
49 devices found in 0.1 seconds.
49 devices found in 0.15 seconds.
49 devices found in 0.2 seconds.
49 devices found in 0.25 seconds.
49 devices found in 0.3 seconds.
49 devices found in 0.35 seconds.
49 devices found in 0.4 seconds.
49 devices found in 0.45 seconds.
49 devices found in 0.5 seconds.
49 devices found in 0.55 seconds.
49 devices found in 0.6 seconds.
49 devices found in 0.65 seconds.
49 devices found in 0.7 seconds.
49 devices found in 0.75 seconds.
49 devices found in 0.8 seconds.
49 devices found in 0.85 seconds.
49 devices found in 0.9 seconds.
50 devices found in 0.95 seconds.


In [14]:
BleakModel.bt_devices

{'89C1A686-1457-77B8-2D9F-22D2EEB7D63C': BLEDevice(89C1A686-1457-77B8-2D9F-22D2EEB7D63C, None),
 'A4B5822F-B0E7-B780-2294-709B891C4D17': BLEDevice(A4B5822F-B0E7-B780-2294-709B891C4D17, SNUG_smartVentilator),
 'E9A5B39F-14EE-784C-16E4-002303200250': BLEDevice(E9A5B39F-14EE-784C-16E4-002303200250, OpenFit by Shokz),
 '14863C4D-BF71-4EA3-C6B4-98001056AAF8': BLEDevice(14863C4D-BF71-4EA3-C6B4-98001056AAF8, WHOOPDEDOO),
 '385E107F-4647-637F-E5DA-39725C39B106': BLEDevice(385E107F-4647-637F-E5DA-39725C39B106, None),
 '867F3213-C9A9-F661-47F0-607135923369': BLEDevice(867F3213-C9A9-F661-47F0-607135923369, 20272A03713613AAD),
 '6A49E2AF-A64F-6674-BFAE-DFFE77CEA51C': BLEDevice(6A49E2AF-A64F-6674-BFAE-DFFE77CEA51C, LEDDMX-00-044B),
 'DB97A830-B1F2-DF91-8381-42C05FC00C40': BLEDevice(DB97A830-B1F2-DF91-8381-42C05FC00C40, None),
 '2AC8DE9A-46FF-3D9A-C132-F528387E7238': BLEDevice(2AC8DE9A-46FF-3D9A-C132-F528387E7238, None),
 '0F40B3D6-A9E7-6552-89C2-B2A88BD38202': BLEDevice(0F40B3D6-A9E7-6552-89C2-B2A8

Copy-and-paste the key corresponding to the device that you want to connect to:

In [15]:
target_address = "14863C4D-BF71-4EA3-C6B4-98001056AAF8"

Or search for the name of the device from the values and retrieve its key.

While most consumer heart rate monitors have a name, this name might be not unique or missing.

In [16]:
target_name = "WHOOPDEDOO"

target_address = ""
for k,v in BleakModel.bt_devices.items():
    if v.name == target_name:
        target_address = k

Anyhow, we will attempt to connect to the following address:

In [17]:
print(target_address)

14863C4D-BF71-4EA3-C6B4-98001056AAF8


In [18]:
# The sleeps are in here to give the scanner time to find the device,
# if and when you "Run All" this jupyter notebook.
await asyncio.sleep(3)
await model.set_target(target_address)

True

When you provide a target address to the `model`, its state changes to:

In [19]:
model.state

'TargetSet'

From `TargetSet` state, you may now attempt to connect to the device.

By default, if it has been over 120 seconds since the last scan, if `auto_rescan=True`, the model will automatically re-scan and then attempt the connection.
These parameters may be changed in the `BleakModule` instantiation above.

In [20]:
await model.connect()

Connected to 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

In [21]:
model.state

'Connected'

In order to receive/send actual data from/to the connected device, we need to start a stream.

This will run forever until `model.disconnect()` is called, so for now let's sleep for several seconds and then call that.

In [22]:
async def run_stream():
    await model.stream()

asyncio.create_task(run_stream())

<Task pending name='Task-14' coro=<run_stream() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_34388/3697076873.py:1>>

In [23]:
await asyncio.sleep(5)

Started streaming
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart Rate: 75 beats per minute
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart Rate: 75 beats per minute
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart Rate: 75 beats per minute
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart Rate: 75 beats per minute
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart Rate: 75 beats per minute


Bleak-FSM encapsulates a kind of bug in Bleak, where it is not possible to re-use a connected client. 

That is, we shouldn't step back to "Connected" state from "Streaming", because we can't go from "Connected" back to "Streaming".

Instead we need to go back two steps to "TargetSet", re-establish a connection, and then we can stream again.

Bleak-FSM enforces this behavior for your own safety.
There exists not method for you to even try to go from "Streaming" to "Connected".


In [24]:
await model.disconnect()

Stopped streaming
Disconnected from 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

In [25]:
model.state

'TargetSet'

In [26]:
await model.unset_target()

True

In [27]:
model.state

'Init'

At this point, we have returned to 'Init' state, and you may 'start_scan', 'set_target_address', 'connect', etc.

# Again, with Errors

Let's review the above tutorial, this time also checking that the proper errors are being emitted when you  attempt illegal things.

We'll discuss how best to handle those errors.

Removing some safeguards by setting `auto_rescan=False`. The default is True.

In [28]:
model = BleakModel(
    scan_stale_time=5, # making it short for the sake of the example
    auto_rescan=False,
)

machine.add_model(model)

In [29]:
model.state

'Init'

Let's try to skip a step in the Finite State Machine.

From `Init`, we are only allowed to go to `TargetSet`.

If we try to go to `Connect`, we should get a `MachineError`:

In [30]:
from transitions import MachineError
try:
    await model.connect()
except MachineError as e:
    print("Correct error raised:")
    print(e)

Correct error raised:
"Can't trigger event connect from state Init!"


In order to go to `TargetSet`, we need an address.

To get an address, we need to scan. For simplicity, we'll just `await` it instead of sending it to the background as we did above.

In [31]:
await model.start_scan()

True

This puts us in "Scaning" state:

In [32]:
print(model.state)

Scanning


From "Scanning", transitioning to any other state besides "Init" is illegal:

In [33]:
try:
    await model.connect()
except MachineError as e:
    print("Correct error raised:")
    print(e)

Correct error raised:
"Can't trigger event connect from state Scanning!"


Let's stop the scan.

In [34]:
await asyncio.sleep(3)
await model.stop_scan()

True

The fact that `start_scan()` and `stop_scan()` are separated out like this means that we can do interesting things, like running a scan until a specified number of devices are found:

In [35]:
print(model.state)

Init


As before, when scanning is done, the results are saved in the class variable `BleakModel.bt_devices`.

Let's throw a wrench into the gears and use an invalid address.
We expect a `BleakFSMError`

In [36]:
from bleak_fsm import BleakFSMError

In [37]:
incorrect_target_address = "some_incorrect_nonexistent_address"
try:
    await model.set_target(incorrect_target_address)
except BleakFSMError as e:
    print("Correct error raised:")
    print(e)

Correct error raised:
Address some_incorrect_nonexistent_address not found in discovered devices


Now back on track, let's set the target with a correct target address determined above:

In [38]:
await model.set_target(target_address)

True

We will now test what happens if we try to connect when the last scan is too old. 

Notice how we instantiated `model` as:
```
model = BleakModel(
    scan_stale_time=5,
    auto_rescan=False,
)
```

If we sleep 5 seconds, then the scanned results will be considered stale. And since `auto_rescan` was set to False, we will get a `StaleScanError`:

In [39]:
from bleak_fsm import StaleScanError

In [40]:
await asyncio.sleep(5) # sleeping in order to trigger stale scan error

try:
    await model.connect()
except StaleScanError as e:
    print("Correct error raised:")
    print(e)


Correct error raised:
Scan is stale. Please rescan. Consider setting 'auto_rescan=True' to avoid this error.


The error message gives us ways to handle this error.

Let's rescan:

In [41]:
await model.unset_target()
await model.start_scan()
await asyncio.sleep(3)
await model.stop_scan()

# then we can connect as long as we do it immediately
await model.set_target(target_address)
await model.connect()

Connected to 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

In [42]:
model.state

'Connected'

In [43]:
await model.disconnect()

Disconnected from 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

In [44]:
await model.unset_target()

True

In [45]:
model.state

'Init'